In [1]:
from gait_analysis import CasiaDataset, settings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
from torch.utils.data import DataLoader
from torchvision import utils

warnings.filterwarnings('ignore')

%matplotlib inline  


# Configure a dataset aggregation of input/labels

Definition of the datasets and the indexing

In [2]:
from gait_analysis.Config import Config
c = Config()
c.config['indexing']['grouping'] = 'person_sequence'
c.config['pose']['load'] = True
c.config['flow']['load'] = True
c.config['heatmaps']['load'] = True



loading configuration  cnn_flows_pretrain
[OK]


Define the dataset_output 

In [9]:
c.config['dataset_output'] = {
        'data': ["scenes","flows","heatmaps"],
        'label': "annotations"}
c.config['scenes']['load']=True
c.config['flow']['load']=True

In [10]:
c.config['transformers'] = {
        # 'Crop':{'include list':['LAnkle','RAnkle'],'output_size':256,'target':'flows'}
        'SpanImagesList': {'remove':True, 'names': ["heatmaps_LAnkle","heatmaps_RAnkle"],'target': ["heatmaps"]},
        'Rescale': {'output_size' : (640,480), 'target': ["heatmaps_LAnkle","heatmaps_RAnkle"]},
        'AnnotationToLabel': {'target': ["annotations"]},
        'Transpose': {'swapping': (2, 0, 1), 'target': ["scenes", "flows"]},
#         'Transpose' : {'swapping': (2, 0, 1) , 'target': ["scenes"]},
        'DimensionResize' : {'dimension': 40, 'target': ["heatmaps_LAnkle","heatmaps_RAnkle","scenes","flows","annotations"]},
#         'DimensionResize' : {'dimension': 10, 'target': ["scenes","annotations"]},
        'ToTensor': {'target':["scenes","annotations"]}
    }

Plain and untransformed dataset

In [11]:
dataset_plain = CasiaDataset()

In [12]:
item, labels = dataset_plain[1]
scenes = item['scenes']
flows = item['flows']
print(item.keys())

dict_keys(['scenes', 'flows', 'heatmaps'])


# Using the composer to construct reusable transformers

In [13]:
from gait_analysis import Composer

composer = Composer()
transformer = composer.compose()
print(transformer)

In [14]:
print(c.get_indexing_grouping())
test_item, labels_df = dataset_plain[0]

person_sequence


just to test we include add a field annotations 


In [15]:
test_item['annotations'] = labels_df
transformed_item = transformer(test_item)
print('==> transformed item is: ',test_item.keys())

==> transformed item is:  dict_keys(['scenes', 'flows', 'annotations', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])


In [16]:
item.keys()

dict_keys(['scenes', 'flows', 'heatmaps'])

In [17]:
len(transformed_item['scenes'])

40

In [20]:
labels_df

{'': ['30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81'],
 'frame_id': ['30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81'],
 'left_foot': ['IN_THE_AIR',
  'IN_THE_AIR',
  'IN_THE_AIR',
  'IN_THE_AIR',
  'IN_THE_AIR',
  'IN_THE_AIR',
  'ON_GROUND',
  'ON_GROUND',
  'ON_GROUN

# Using the transformer and the dataset_output specifications

Printing some samples of the transformed dataset

In [21]:
c.config['dataset_output'] = {
        'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'label': "annotations"}
transformed_dataset = CasiaDataset(transform=transformer)

for i in range(len(transformed_dataset)):
    print('i: ', i)
    sample, labels = transformed_dataset[i]

    print(i, sample.keys())
    print(labels.size())

    if i == 5:
        break

i:  0
0 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])
i:  1
1 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])
i:  2
2 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])
i:  3
3 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])
i:  4
4 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])
i:  5
5 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([40])


# Dataloader: using the transformed dataset with the transformer to create a training set

In [14]:
# dataloader = DataLoader(transformed_dataset, batch_size=4,
#                         shuffle=True, num_workers=4)
dataloader = DataLoader(transformed_dataset, batch_size=10)

#### DEV: The problem is that the dimensions of the label are different for some of the videos. In this case the aggregation works for all the angles in the video but no arbitrary batch_size will work

In [22]:
def plot_flows(inputs):
    flows = inputs['flows']
    print('plotting sample flow chunck')
    flow = flows[4]
    batch_size = flow.size(0)
    im_size = flow.size()
    print('im_size = ', im_size)
    print('batch_size = ', batch_size )
    grid = utils.make_grid(flow)
    plt.figure(figsize=(10,50))
    plt.imshow(grid.numpy()[2,:,:])
    plt.show()
    
def plot_scenes(inputs):
    scenes = inputs['scenes']
    print('plotting sample scene chunck')
    scene = scenes[4]
    grid = utils.make_grid(scene)
    plt.figure(figsize=(10,50))
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.show()
    
def plot_heatmap(inputs):
    hmaps = inputs['heatmaps_LAnkle']
    print('plotting sample heatmasp chunck')
    hmap = hmaps[4].numpy()
    print('hmap.shape = ',hmap.shape)
    plt.figure(figsize=(10,50))
    plt.subplot(151)
    plt.imshow(hmap[1,:,:])
    plt.subplot(152)
    plt.imshow(hmap[2,:,:])
    plt.subplot(153)
    plt.imshow(hmap[3,:,:])
    plt.subplot(154)
    plt.imshow(hmap[4,:,:])
    plt.subplot(155)
    plt.imshow(hmap[5,:,:])
    

    
    
    
for i_batch, sample_batched in enumerate(dataloader):
    # observe 4th batch and stop.
    if i_batch == 9:
        inputs, labels = sample_batched
        plot_flows(inputs)
        plot_scenes(inputs)
        plot_heatmap(inputs)
        print(type(sample_batched))
        print('labels,',labels.size(),labels)
        break

In [16]:
a = np.eye(400)

a.resize((1,400,400))
a.shape

(1, 400, 400)

In [21]:
i,l = transformed_dataset[42]
print(l)

IndexError: list index out of range